In [1]:
import glob
import os
from sklearn.preprocessing import LabelEncoder

In [1]:
print("1")

1


In [ ]:
print(2)

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


In [7]:
with open("WSJCorpus\pos\wsj_0005.pos", "r") as x:
    y = x.read()
    print(y.split())

['J.P./NNP', 'Bolduc/NNP', ',/,', 'vice/NN', 'chairman/NN', 'of/IN', 'W.R./NNP', 'Grace/NNP', '&/CC', 'Co./NNP', ',/,', 'which/WDT', 'holds/VBZ', 'a/DT', '83.4/CD', '%/NN', 'interest/NN', 'in/IN', 'this/DT', 'energy-services/JJ', 'company/NN', ',/,', 'was/VBD', 'elected/VBN', 'a/DT', 'director/NN', './.', 'He/PRP', 'succeeds/VBZ', 'Terrence/NNP', 'D./NNP', 'Daniels/NNP', ',/,', 'formerly/RB', 'a/DT', 'W.R./NNP', 'Grace/NNP', 'vice/NN', 'chairman/NN', ',/,', 'who/WP', 'resigned/VBD', './.', 'W.R./NNP', 'Grace/NNP', 'holds/VBZ', 'three/CD', 'of/IN', 'Grace/NNP', 'Energy/NNP', "'s/POS", 'seven/CD', 'board/NN', 'seats/NNS', './.']


In [8]:
total_tags = []
sentences = []
input_folder = glob.glob(r'WSJCorpus\pos\*')

for pos_file in input_folder:
    try:
        with open(pos_file, 'r') as file:
            # Read the contents of the file
            pos_data = file.read()
            a = pos_data.split()
            temp_words = []
            temp_tags = []
            for i in a:
                temp_words.append(i.rsplit("/",1)[0].lower())
                temp_tags.append(i.rsplit("/",1)[1])
                if i[-1] == "." or len(temp_words) == 15:  # maximum length of sentence is 15  any sentence more than 15 length is shortened to 15.
                    sentences.append(temp_words)
                    total_tags.append(temp_tags)
                    temp_words, temp_tags = [],[]
    except FileNotFoundError:
        print(f"File '{pos_file}' not found.")

In [9]:
total_words = [i for sublist in sentences for i in sublist]
flatten_tags = [i for sublist in total_tags for i in sublist]
unique_words = set(total_words)
unique_tags = set(flatten_tags)

print("total number of sentences are", len(sentences))
print("total number of words are",len(total_words))
print("total number of tags are",len(flatten_tags))
print("total unique words are",len(unique_words))
print("total unique tags are",len(unique_tags))


total number of sentences are 8081

total number of words are 94156

total number of tags are 94156

total unique words are 10955

total unique tags are 46


In [17]:
from transformers import BertModel, BertTokenizer

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Convert words into input IDs using the tokenizer
input_ids = [tokenizer.convert_tokens_to_ids(words) for words in sentences]

# Pad sequences to the same length
max_length = max(len(ids) for ids in input_ids)
padded_input_ids = [ids + [tokenizer.pad_token_id] * (max_length - len(ids)) for ids in input_ids]

# Convert lists of input IDs to PyTorch tensors
input_tensors = torch.tensor(padded_input_ids)

# Forward pass through the BERT model to obtain contextualized embeddings
with torch.no_grad():
    outputs = model(input_tensors)

# Get the contextualized word embeddings (output of the last layer)
last_hidden_states = outputs.last_hidden_state

# Print the shape of the contextualized word embeddings
print("Shape of contextualized word embeddings:", last_hidden_states.shape)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\win\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\win\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.

To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development

  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [335]:
# 1. Download and load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

# Adjust the file path to point to your downloaded GloVe embeddings file
glove_file = 'glove.6B\glove.6B.50d.txt'
glove_embeddings = load_glove_embeddings(glove_file)

# 3. Get word vectors for a list of words
word_vectors = {}
no_word_vectors = []
for word in unique_words:
    if word in glove_embeddings:
        word_vectors[word] = glove_embeddings[word]
    else:
        no_word_vectors.append(word)

In [336]:
base_array = np.zeros(50,dtype = np.float64)
for word in no_word_vectors:
    word_vectors[word] = base_array

In [10]:
# sentence_lengths = [len(i) for i in sentences]
# sentence_lengths.sort(reverse=True)
# sentence_lengths[:10]


[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]

#### remove sentences with length more than 80

In [338]:
# to_be_removed = []
# count = 0
# for i in sentences:
#     if len(i)>80:
#         to_be_removed.append(count)
#     count+=1


In [341]:
# count = 0
# for i in to_be_removed:
#     sentences.pop(i-count)
#     # total_tags.pop(i-count)
#     count+=1

In [342]:
# len(sentences), len(total_tags)

(3869, 3869)

In [11]:
# sentence_lengths = [len(i) for i in sentences]
# sentence_lengths.sort(reverse=True)
# sentence_lengths[:10]


[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]

In [351]:
# Label Encoding for tags
unique_tags.add("no_word")
le = LabelEncoder()
le.fit(list(unique_tags))
total_tags_y = [le.transform(i) for i in total_tags]
len(total_tags_y)

3869

In [366]:
le.inverse_transform([46])

array(['no_word'], dtype='<U7')

In [345]:
sentence_vectors = []

for sentence in sentences:
    temp_sentence = []
    for word in sentence:
        temp_sentence.append(word_vectors[word])
    sentence_vectors.append(temp_sentence)
len(sentence_vectors)


3869

In [352]:
# Convert word-level labels to one-hot encoded format
num_classes = len(unique_tags)  # Number of classes in your target variable
word_labels_one_hot = [to_categorical(labels, num_classes=num_classes) for labels in total_tags_y]

In [448]:
no_word_vector = to_categorical(le.transform(["no_word"]))
no_word_vector

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],
      dtype=float32)

In [450]:
repeated_a = np.repeat(no_word_vector, 2, axis=0).reshape(-1, no_word_vector.shape[1])
repeated_a

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],
      dtype=float32)

In [453]:
def pad_sequence(sentences,max_length,word_dim):
    sentence_vectors_padded = []
    word_labels_one_hot_padded = []
    for sentence in sentences:
        to_be_added = max_length - len(sentence)
        # print(np.zeros((to_be_added,word_dim)).shape)
        # print(np.array(sentence).shape)
        temp_array = np.vstack((np.zeros((to_be_added,word_dim)),np.array(sentence)))

        sentence_vectors_padded.append(temp_array)

    sentence_vectors_padded = np.array(sentence_vectors_padded)
    return sentence_vectors_padded

def pad_sequence_labels(word_labels_one_hot,max_length,label_dim):
    word_labels_one_hot_padded = []
    for sentence in word_labels_one_hot:
        to_be_added = max_length - len(sentence)
        # print(np.zeros((to_be_added,word_dim)).shape)
        # print(np.array(sentence).shape)
        repeated_a = np.repeat(no_word_vector, to_be_added, axis=0).reshape(-1, no_word_vector.shape[1])

        temp_array = np.vstack((repeated_a,np.array(sentence)))

        word_labels_one_hot_padded.append(temp_array)

    word_labels_one_hot_padded = np.array(word_labels_one_hot_padded)
    return word_labels_one_hot_padded


In [454]:
sentence_vectors_padded = pad_sequence(sentence_vectors, 80, 50)
word_labels_one_hot_padded = pad_sequence_labels(word_labels_one_hot,80,47)

In [455]:
X = np.array(sentence_vectors_padded)
y = np.array(word_labels_one_hot_padded)

# Split the dataset into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

In [457]:
hidden_units = 50
max_length = 80
num_features = 50

model = Sequential([
    Masking(mask_value=0.0, input_shape=(max_length, num_features)),
    LSTM(units=hidden_units, return_sequences=True),
    Dropout(0.2),  # Add dropout for regularization
    Dense(units=num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])


Epoch 1/50

73/73 [==============================] - 23s 121ms/step - loss: 3.1923 - accuracy: 0.1618 - val_loss: 2.7376 - val_accuracy: 0.2807

Epoch 2/50

73/73 [==============================] - 5s 67ms/step - loss: 2.4682 - accuracy: 0.3514 - val_loss: 2.1539 - val_accuracy: 0.4379

Epoch 3/50

73/73 [==============================] - 4s 55ms/step - loss: 1.9791 - accuracy: 0.4749 - val_loss: 1.7611 - val_accuracy: 0.5375

Epoch 4/50

73/73 [==============================] - 4s 61ms/step - loss: 1.6554 - accuracy: 0.5667 - val_loss: 1.4822 - val_accuracy: 0.6332

Epoch 5/50

73/73 [==============================] - 5s 64ms/step - loss: 1.4256 - accuracy: 0.6332 - val_loss: 1.2876 - val_accuracy: 0.6779

Epoch 6/50

73/73 [==============================] - 7s 96ms/step - loss: 1.2680 - accuracy: 0.6678 - val_loss: 1.1587 - val_accuracy: 0.7028

Epoch 7/50

73/73 [==============================] - 4s 53ms/step - loss: 1.1605 - accuracy: 0.6901 - val_loss: 1.0660 - val_accuracy: 0.718

In [458]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

25/25 [==============================] - 1s 19ms/step - loss: 0.4488 - accuracy: 0.8627

Test Loss: 0.4488038420677185

Test Accuracy: 0.8626728653907776


In [468]:
y_test_pred

array([[[2.1120718e-02, 1.6701329e-02, 2.1840518e-02, ...,
         1.8913593e-02, 1.8918350e-02, 1.7702667e-02],
        [2.1120718e-02, 1.6701329e-02, 2.1840518e-02, ...,
         1.8913593e-02, 1.8918350e-02, 1.7702667e-02],
        [2.1120718e-02, 1.6701329e-02, 2.1840518e-02, ...,
         1.8913593e-02, 1.8918350e-02, 1.7702667e-02],
        ...,
        [3.4472141e-05, 1.3238936e-05, 6.1052994e-08, ...,
         1.0707229e-07, 2.1313971e-08, 4.4602034e-06],
        [3.5439582e-05, 2.6861574e-06, 1.4485667e-07, ...,
         1.2273174e-05, 2.2856040e-07, 4.4837787e-05],
        [1.3607648e-05, 7.6095253e-06, 1.8028872e-05, ...,
         5.4443412e-04, 2.2154433e-07, 2.8998281e-06]],

       [[2.1120718e-02, 1.6701329e-02, 2.1840518e-02, ...,
         1.8913593e-02, 1.8918350e-02, 1.7702667e-02],
        [2.1120718e-02, 1.6701329e-02, 2.1840518e-02, ...,
         1.8913593e-02, 1.8918350e-02, 1.7702667e-02],
        [2.1120718e-02, 1.6701329e-02, 2.1840518e-02, ...,
         1.891

In [459]:
y_test_pred = model.predict(X_test)

25/25 [==============================] - 2s 15ms/step


In [460]:
# Get the index of the class with the highest probability for each word
predicted_tags_indices = np.argmax(y_test, axis=-1)  # Shape: (1, 80)

actual_tags = [le.inverse_transform(i) for i in predicted_tags_indices]

# Now predicted_tags contains the predicted tag labels for each word in the sentence
# You can access predicted_tags[0] to get the predicted tags for the entire sentence
print(actual_tags)

[array(['no_word', 'no_word', 'no_word', 'no_word', 'no_word', 'no_word',

       'no_word', 'no_word', 'no_word', 'no_word', 'no_word', 'no_word',

       'no_word', 'no_word', 'no_word', 'no_word', 'no_word', 'no_word',

       'no_word', 'no_word', 'no_word', 'no_word', 'no_word', 'no_word',

       'no_word', 'no_word', 'no_word', 'no_word', 'no_word', 'IN', 'CD',

       'NN', 'IN', 'DT', 'NN', 'NN', 'VBP', 'DT', 'NNS', 'IN', 'NN', 'IN',

       'NNP', 'NNP', ':', 'DT', 'NNP', 'NNP', ':', 'VBG', 'IN', 'JJ',

       'NN', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'NNS', ',', 'PRP$', 'NN',

       'NN', 'NNS', 'IN', 'NN', 'NNS', 'IN', 'NN', ',', 'CC', 'DT', 'NNS',

       'IN', 'DT', 'JJ', 'NNP', 'NNS', 'NNS', '.'], dtype='<U7'), array(['no_word', 'no_word', 'no_word', 'no_word', 'no_word', 'no_word',

       'no_word', 'no_word', 'no_word', 'no_word', 'no_word', 'no_word',

       'no_word', 'no_word', 'no_word', 'no_word', 'no_word', 'no_word',

       'no_word', 'no_word', 'no_word', 'no_wo

In [461]:
# Get the index of the class with the highest probability for each word
predicted_tags_indices = np.argmax(y_test_pred, axis=-1)  # Shape: (1, 80)

predicted_tags = [le.inverse_transform(i) for i in predicted_tags_indices]

# Now predicted_tags contains the predicted tag labels for each word in the sentence
# You can access predicted_tags[0] to get the predicted tags for the entire sentence
print(predicted_tags)

[array(['NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'IN', 'DT', 'JJ', 'IN', 'DT', 'NN', 'NN', 'NN', 'DT',

       'NNS', 'IN', 'VBG', 'IN', 'NNP', 'NNP', ':', 'DT', 'NNP', 'NNP',

       ':', 'JJ', 'IN', 'JJ', 'NN', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'NNS',

       ',', 'PRP$', 'NN', 'NN', 'NNS', 'IN', 'JJ', 'NNS', 'IN', 'NN', ',',

       'CC', 'DT', 'NNS', 'IN', 'DT', 'JJ', 'NNP', 'NNS', 'NNS', '.'],

      dtype='<U7'), array(['NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'NNP', 'NNP', 'NNP

In [471]:
actual_tags[0]

array(['no_word', 'no_word', 'no_word', 'no_word', 'no_word', 'no_word',
       'no_word', 'no_word', 'no_word', 'no_word', 'no_word', 'no_word',
       'no_word', 'no_word', 'no_word', 'no_word', 'no_word', 'no_word',
       'no_word', 'no_word', 'no_word', 'no_word', 'no_word', 'no_word',
       'no_word', 'no_word', 'no_word', 'no_word', 'no_word', 'IN', 'CD',
       'NN', 'IN', 'DT', 'NN', 'NN', 'VBP', 'DT', 'NNS', 'IN', 'NN', 'IN',
       'NNP', 'NNP', ':', 'DT', 'NNP', 'NNP', ':', 'VBG', 'IN', 'JJ',
       'NN', 'NNS', 'IN', 'DT', 'JJ', 'NN', 'NNS', ',', 'PRP$', 'NN',
       'NN', 'NNS', 'IN', 'NN', 'NNS', 'IN', 'NN', ',', 'CC', 'DT', 'NNS',
       'IN', 'DT', 'JJ', 'NNP', 'NNS', 'NNS', '.'], dtype='<U7')

In [473]:
i = 0
count = 0
while i==len(actual_tags):
    if actual_tags[i].all == predicted_tags[i].all:
        count+=1
    i+=1

In [474]:
count

0

In [462]:
a = 'i am playing game .'
b = a.split()

In [463]:
c = []
for i in b:
    c.append(word_vectors[i])

In [464]:
to_be_added = max_length - len(c)
# print(np.zeros((to_be_added,word_dim)).shape)
# print(np.array(sentence).shape)
d = np.vstack((np.zeros((to_be_added,50)),np.array(c)))


In [465]:
# Reshape the sentence to match the expected input shape of the model
# Since we have only one sentence, we add an extra dimension to represent the batch size
input_sentence = d[np.newaxis, :, :]  # Shape: (1, 80, 50)

# Now, you can use model.predict() to obtain predictions for this sentence
predictions = model.predict(input_sentence)

# Do something with the predictions
print(predictions)

1/1 [==============================] - 0s 213ms/step

[[[2.1120718e-02 1.6701329e-02 2.1840518e-02 ... 1.8913593e-02

   1.8918350e-02 1.7702667e-02]

  [2.1120718e-02 1.6701329e-02 2.1840518e-02 ... 1.8913593e-02

   1.8918350e-02 1.7702667e-02]

  [2.1120718e-02 1.6701329e-02 2.1840518e-02 ... 1.8913593e-02

   1.8918350e-02 1.7702667e-02]

  ...

  [7.7547261e-04 2.3660436e-05 6.6928311e-05 ... 2.3564845e-03

   1.3210117e-04 1.9487849e-04]

  [5.1310140e-04 1.6411269e-04 1.0366436e-04 ... 8.6252834e-04

   1.2717550e-04 3.3406654e-05]

  [1.8767383e-05 7.8689191e-06 2.8646921e-06 ... 3.8393488e-04

   5.2641678e-08 1.6018454e-06]]]


In [466]:
# Get the index of the class with the highest probability for each word
predicted_tags_indices = np.argmax(predictions, axis=-1)  # Shape: (1, 80)

predicted_tags = [le.inverse_transform(i) for i in predicted_tags_indices]

# Now predicted_tags contains the predicted tag labels for each word in the sentence
# You can access predicted_tags[0] to get the predicted tags for the entire sentence
print(predicted_tags)

[array(['NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP', 'NNP',

       'NNP', 'NNP', 'NNP', 'PRP', 'PRP', 'RB', 'JJ', '.'], dtype='<U7')]
